Collaborative Filtering

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
from collections import defaultdict
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from sklearn.preprocessing import normalize
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/amanparikh/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
anime = pd.read_csv('anime.csv')
user = pd.read_csv('user_anime_list.0.csv')

In [3]:
anime.head()

,anime_id,title,main_picture,alternative_titles,start_date,end_date,synopsis,num_list_users,num_scoring_users,nsfw,medium,status,num_episodes,start_season,source,average_episode_duration,studios,related_anime,recommendations,genres
0,1,Cowboy Bebop,{'medium': 'https://api-cdn.myanimelist.net/im...,"{'synonyms': [], 'en': 'Cowboy Bebop', 'ja': '...",1998-04-03,1999-04-24,"Crime is timeless. By the year 2071, humanity ...",1731562,892212,white,tv,finished_airing,26,"{'year': 1998, 'season': 'spring'}",original,1440,"[{'id': 14, 'name': 'Sunrise'}]","[{'anime_id': 5, 'relation': 'side_story'}, {'...","[{'anime_id': 205, 'num_recommendations': 118}...","['Action', 'Adult Cast', 'Award Winning', 'Sci..."
1,5,Cowboy Bebop: Tengoku no Tobira,{'medium': 'https://api-cdn.myanimelist.net/im...,"{'synonyms': [""Cowboy Bebop: Knockin' on Heave...",2001-09-01,2001-09-01,"Another day, another bounty—such is the life o...",353984,202526,white,movie,finished_airing,1,"{'year': 2001, 'season': 'summer'}",original,6911,"[{'id': 4, 'name': 'Bones'}]","[{'anime_id': 1, 'relation': 'parent_story'}]","[{'anime_id': 4106, 'num_recommendations': 3},...","['Action', 'Adult Cast', 'Sci-Fi', 'Space']"
2,6,Trigun,{'medium': 'https://api-cdn.myanimelist.net/im...,"{'synonyms': [], 'en': 'Trigun', 'ja': 'トライガン'}",1998-04-01,1998-09-30,"Vash the Stampede is the man with a $$60,000,0...",707864,347350,white,tv,finished_airing,26,"{'year': 1998, 'season': 'spring'}",manga,1480,"[{'id': 11, 'name': 'Madhouse'}]","[{'anime_id': 4106, 'relation': 'side_story'},...","[{'anime_id': 1, 'num_recommendations': 85}, {...","['Action', 'Adult Cast', 'Adventure', 'Sci-Fi'..."
3,7,Witch Hunter Robin,{'medium': 'https://api-cdn.myanimelist.net/im...,"{'synonyms': ['WHR'], 'en': 'Witch Hunter Robi...",2002-07-03,2002-12-25,Robin Sena is a powerful craft user drafted in...,110305,42443,white,tv,finished_airing,26,"{'year': 2002, 'season': 'summer'}",original,1500,"[{'id': 14, 'name': 'Sunrise'}]",[],"[{'anime_id': 2025, 'num_recommendations': 12}...","['Action', 'Detective', 'Drama', 'Mystery', 'S..."
4,8,Bouken Ou Beet,{'medium': 'https://api-cdn.myanimelist.net/im...,"{'synonyms': ['Adventure King Beet'], 'en': 'B...",2004-09-30,2005-09-29,It is the dark century and the people are suff...,14822,6353,white,tv,finished_airing,52,"{'year': 2004, 'season': 'fall'}",manga,1380,"[{'id': 18, 'name': 'Toei Animation'}]","[{'anime_id': 1123, 'relation': 'sequel'}]","[{'anime_id': 417, 'num_recommendations': 2}, ...","['Adventure', 'Fantasy', 'Shounen', 'Supernatu..."


In [4]:
#anime.count()

In [5]:
user.head()

,uid,status,score,num_episodes_watched,is_rewatching,start_date,finish_date,priority,num_times_rewatched,rewatch_value,updated_at,username
0,6408,plan_to_watch,0,0,False,NaN,NaN,-1,-1,-1,2022-03-16T23:49:49+00:00,Imfckingsleepy
1,52034,plan_to_watch,0,0,False,NaN,NaN,-1,-1,-1,2022-06-18T22:15:01+00:00,Imfckingsleepy
2,41380,completed,6,12,False,NaN,NaN,-1,-1,-1,2022-03-19T01:50:33+00:00,Imfckingsleepy
3,44881,completed,7,12,False,NaN,2022-03-19,-1,-1,-1,2022-03-19T13:55:35+00:00,Imfckingsleepy
4,35248,plan_to_watch,0,0,False,NaN,NaN,-1,-1,-1,2022-08-28T20:05:54+00:00,Imfckingsleepy


In [6]:
user.count()

uid                     7360134
status                  7360127
score                   7360134
num_episodes_watched    7360134
is_rewatching           7360134
start_date              1225783
finish_date             1560725
priority                7360134
num_times_rewatched     7360134
rewatch_value           7360134
updated_at              7360134
username                7360134
dtype: int64

In [7]:
user.rename(columns={"uid":"anime_id"},inplace=True)

In [8]:
user.status.unique()

array(['plan_to_watch', 'completed', 'dropped', 'watching', 'on_hold',
       nan], dtype=object)

In [9]:
user_cleaned=user[user.status!="plan_to_watch"]

In [10]:
user_cleaned=user_cleaned[user_cleaned.score!=0]

In [11]:
user_cleaned.count()

anime_id                3810865
status                  3810863
score                   3810865
num_episodes_watched    3810865
is_rewatching           3810865
start_date               879110
finish_date             1283776
priority                3810865
num_times_rewatched     3810865
rewatch_value           3810865
updated_at              3810865
username                3810865
dtype: int64

In [12]:
user_cleaned=user_cleaned[['anime_id','username','score']]

In [13]:
user_cleaned.head()

,anime_id,username,score
2,41380,Imfckingsleepy,6
3,44881,Imfckingsleepy,7
5,31646,Imfckingsleepy,9
7,38101,Imfckingsleepy,6
8,39783,Imfckingsleepy,3


In [14]:
anime_id = anime['anime_id']
anime_id

0            1
1            5
2            6
3            7
4            8
         ...  
11294    54638
11295    54643
11296    54644
11297    54650
11298    54660
Name: anime_id, Length: 11299, dtype: int64

In [15]:
user_cleaned = user_cleaned[user_cleaned['anime_id'].isin(anime_id)]

In [16]:
user_cleaned.head()

,anime_id,username,score
5,31646,Imfckingsleepy,9
7,38101,Imfckingsleepy,6
9,41457,Imfckingsleepy,9
12,42847,Imfckingsleepy,8
19,6547,Imfckingsleepy,9


In [17]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()

In [18]:
user_cleaned['user_id']=le.fit_transform(user_cleaned.username)

In [19]:
user_cleaned['extra']=1

In [20]:
user_cleaned.shape

(1782688, 5)

In [21]:
tryt=user_cleaned.groupby('user_id', as_index=False).sum()

In [22]:
tryt

,user_id,anime_id,score,extra
0,0,1253489,652,94
1,1,6110297,1579,202
2,2,410411,125,13
3,3,1327307,358,51
4,4,1525495,375,49
...,...,...,...,...
16305,16305,3181981,687,99
16306,16306,391136,117,13
16307,16307,625954,251,32
16308,16308,3790047,829,112


In [23]:
trya=user_cleaned.groupby('anime_id', as_index=False).sum()

In [24]:
trya

,anime_id,score,user_id,extra
0,1,34911,32995405,4119
1,5,10622,10413738,1301
2,6,13263,13218476,1664
3,7,1149,1391430,169
4,8,188,228395,28
...,...,...,...,...
9401,54426,9,24783,2
9402,54427,4,13388,1
9403,54428,5,6342,1
9404,54429,11,17737,2


In [25]:
try_userid=tryt['user_id'][tryt['extra']>50]

In [26]:
try_animeid=trya['anime_id'][trya['extra']>30]

In [27]:
try_animeid

0           1
1           5
2           6
3           7
5          15
        ...  
9265    53414
9275    53525
9278    53581
9285    53599
9354    54026
Name: anime_id, Length: 3603, dtype: int64

In [28]:
user_cleaned.score.value_counts().sum()

1782688

In [29]:
user_cleaned = user_cleaned[user_cleaned['user_id'].isin(try_userid)]

In [30]:
user_cleaned = user_cleaned[user_cleaned['anime_id'].isin(try_animeid)]

In [31]:
user_cleaned.shape

(1589389, 5)

In [32]:
user_cleaned.head()

,anime_id,username,score,user_id,extra
5,31646,Imfckingsleepy,9,4404,1
7,38101,Imfckingsleepy,6,4404,1
9,41457,Imfckingsleepy,9,4404,1
12,42847,Imfckingsleepy,8,4404,1
19,6547,Imfckingsleepy,9,4404,1


In [33]:
user_cleaned.to_csv('user_cleaned.csv')


In [34]:
user_cleaned = pd.read_csv('user_cleaned.csv')
anime = pd.read_csv('anime.csv')

In [35]:
user_cleaned.score.value_counts().sum()

1589389

In [36]:
mergeduseritem = pd.merge(user_cleaned, anime, on ='anime_id')

In [37]:
from sklearn.model_selection import train_test_split

In [38]:
ratings_train,ratings_test = train_test_split(mergeduseritem,test_size=0.2) 

In [39]:
ratings_train.head()

,Unnamed: 0,anime_id,username,score,user_id,extra,title,main_picture,alternative_titles,start_date,...,medium,status,num_episodes,start_season,source,average_episode_duration,studios,related_anime,recommendations,genres
495129,2649474,37349,Big_Neet_Henrik,9,1457,1,Goblin Slayer,{'medium': 'https://api-cdn.myanimelist.net/im...,"{'synonyms': [], 'en': 'Goblin Slayer', 'ja': ...",2018-10-07,...,tv,finished_airing,12,"{'year': 2018, 'season': 'fall'}",light_novel,1420,"[{'id': 314, 'name': 'White Fox'}]","[{'anime_id': 38807, 'relation': 'summary'}, {...","[{'anime_id': 31859, 'num_recommendations': 19...","['Action', 'Adventure', 'Fantasy', 'Gore']"
969432,1053372,11761,ITer,7,4307,1,Medaka Box,{'medium': 'https://api-cdn.myanimelist.net/im...,"{'synonyms': [], 'en': 'Medaka Box', 'ja': 'めだ...",2012-04-05,...,tv,finished_airing,12,"{'year': 2012, 'season': 'spring'}",manga,1440,"[{'id': 6, 'name': 'Gainax'}]","[{'anime_id': 14527, 'relation': 'sequel'}]","[{'anime_id': 849, 'num_recommendations': 7}, ...","['Action', 'Comedy', 'Ecchi', 'Martial Arts', ..."
1164104,5606249,32899,Kinofhera,7,5348,1,Watashi ga Motete Dousunda,{'medium': 'https://api-cdn.myanimelist.net/im...,"{'synonyms': [], 'en': 'Kiss Him, Not Me!', 'j...",2016-10-07,...,tv,finished_airing,12,"{'year': 2016, 'season': 'fall'}",manga,1380,"[{'id': 112, 'name': ""Brain's Base""}]",[],"[{'anime_id': 853, 'num_recommendations': 11},...","['Comedy', 'Otaku Culture', 'Reverse Harem', '..."
865741,6088027,32542,Tekatsuyo,8,9843,1,Sakamoto desu ga?,{'medium': 'https://api-cdn.myanimelist.net/im...,"{'synonyms': ['Sakamoto desu ga?'], 'en': ""Hav...",2016-04-08,...,tv,finished_airing,12,"{'year': 2016, 'season': 'spring'}",manga,1460,"[{'id': 37, 'name': 'Studio Deen'}]","[{'anime_id': 33524, 'relation': 'side_story'}]","[{'anime_id': 32648, 'num_recommendations': 25...","['Comedy', 'Gag Humor', 'School', 'Seinen']"
448249,5513334,27601,Dazonwraith,6,2499,1,Chuunibyou demo Koi ga Shitai! Ren: The Rikka ...,{'medium': 'https://api-cdn.myanimelist.net/im...,{'synonyms': ['Chuunibyou demo Koi ga Shitai! ...,2014-09-17,...,special,finished_airing,1,"{'year': 2014, 'season': 'summer'}",light_novel,1440,"[{'id': 2, 'name': 'Kyoto Animation'}]","[{'anime_id': 18671, 'relation': 'prequel'}]",[],"['Comedy', 'Drama', 'Romance', 'School']"


In [40]:
usersPerItem = defaultdict(set)
itemsPerUser = defaultdict(set)

# Latent factor

In [41]:
import gzip
from collections import defaultdict
import scipy
import scipy.optimize
import numpy
import random

In [42]:
reviewsPerUser = defaultdict(list)
reviewsPerItem = defaultdict(list)

In [43]:
user_lf=ratings_train[['title','username','score']]

In [44]:
user_cleaned = user_lf.to_dict('records')

In [45]:
for d in user_cleaned:
    user,item = d['username'], d['title']
    reviewsPerUser[user].append(d)
    reviewsPerItem[item].append(d)

In [46]:
# N = len(user_cleaned)
# nUsers = len(reviewsPerUser)
# nItems = len(reviewsPerItem)
# users = list(reviewsPerUser.keys())
# items = list(reviewsPerItem.keys())

In [47]:
# ratingMean = sum([d['score'] for d in user_cleaned]) / len(user_cleaned)

In [48]:
# labels = [d['score'] for d in user_cleaned]

In [49]:
# alwaysPredictMean = [ratingMean for d in user_cleaned]

In [50]:
# import scipy.optimize

# Complete Latent factor

In [51]:
# alpha = ratingMean

In [52]:
# userBiases = defaultdict(float)
# itemBiases = defaultdict(float)

In [53]:
# userGamma = {}
# itemGamma = {}

In [54]:
# K = 32

In [55]:
# for u in reviewsPerUser:
#     userGamma[u] = [random.random() * 0.1 - 0.05 for k in range(K)]

In [56]:
# for i in reviewsPerItem:
#     itemGamma[i] = [random.random() * 0.1 - 0.05 for k in range(K)]

In [57]:
# def MSE(predictions, labels):
#     differences = [(x-y)**2 for x,y in zip(predictions,labels)]
#     return sum(differences) / len(differences)

In [58]:
# def unpack(theta):
#     global alpha
#     global userBiases
#     global itemBiases
#     global userGamma
#     global itemGamma
#     index = 0
#     alpha = theta[index]
#     index += 1
#     userBiases = dict(zip(users, theta[index:index+nUsers]))
#     index += nUsers
#     itemBiases = dict(zip(items, theta[index:index+nItems]))
#     index += nItems
#     for u in users:
#         userGamma[u] = theta[index:index+K]
#         index += K
#     for i in items:
#         itemGamma[i] = theta[index:index+K]
#         index += K

In [59]:
def inner(x, y):
    return sum([a*b for a,b in zip(x,y)])

In [60]:
def prediction(user, item):
    return alpha + userBiases[user] + itemBiases[item] + inner(userGamma[user], itemGamma[item])

In [61]:
# def cost(theta, labels, lamb):
#     unpack(theta)
#     predictions = [prediction(d['username'], d['title']) for d in user_cleaned]
#     cost = MSE(predictions, labels)
#     print("MSE = " + str(cost))
#     for u in users:
#         cost += lamb*userBiases[u]**2
#         for k in range(K):
#             cost += lamb*userGamma[u][k]**2
#     for i in items:
#         cost += lamb*itemBiases[i]**2
#         for k in range(K):
#             cost += lamb*itemGamma[i][k]**2
#     return cost

In [62]:
# def derivative(theta, labels, lamb):
#     unpack(theta)
#     N = len(user_cleaned)
#     dalpha = 0
#     dUserBiases = defaultdict(float)
#     dItemBiases = defaultdict(float)
#     dUserGamma = {}
#     dItemGamma = {}
#     for u in reviewsPerUser:
#         dUserGamma[u] = [0.0 for k in range(K)]
#     for i in reviewsPerItem:
#         dItemGamma[i] = [0.0 for k in range(K)]
#     for d in user_cleaned:
#         u,i = d['username'], d['title']
#         pred = prediction(u, i)
#         diff = pred - d['score']
#         dalpha += 2/N*diff
#         dUserBiases[u] += 2/N*diff
#         dItemBiases[i] += 2/N*diff
#         for k in range(K):
#             dUserGamma[u][k] += 2/N*itemGamma[i][k]*diff
#             dItemGamma[i][k] += 2/N*userGamma[u][k]*diff
#     for u in userBiases:
#         dUserBiases[u] += 2*lamb*userBiases[u]
#         for k in range(K):
#             dUserGamma[u][k] += 2*lamb*userGamma[u][k]
#     for i in itemBiases:
#         dItemBiases[i] += 2*lamb*itemBiases[i]
#         for k in range(K):
#             dItemGamma[i][k] += 2*lamb*itemGamma[i][k]
#     dtheta = [dalpha] + [dUserBiases[u] for u in users] + [dItemBiases[i] for i in items]
#     for u in users:
#         dtheta += dUserGamma[u]
#     for i in items:
#         dtheta += dItemGamma[i]
#     return numpy.array(dtheta)

In [63]:
# MSE(alwaysPredictMean, labels)

In [64]:
# scipy.optimize.fmin_l_bfgs_b(cost, [alpha] + # Initialize alpha
#                                    [0.0]*(nUsers+nItems) + # Initialize beta
#                                    [random.random() * 0.1 - 0.05 for k in range(K*(nUsers+nItems))], # Gamma
#                              derivative, args = (labels, 0.001))

In [65]:
# x = []
# for i,j in zip(ratings_test['username'], ratings_test['title']):
#     x.append(prediction(i,j))

In [66]:
# y = list(ratings_test['score'])
# import math as m
# m.sqrt(MSE(y,x))

In [67]:
# d = {}
# d['alpha'] = alpha
# d['userBiases'] = userBiases
# d['itemBiases'] = itemBiases
# d['userGamma'] = userGamma
# d['itemGamma'] = itemGamma

In [68]:
# import pickle 
# with open("D:/D drive/UCSD/New folder/Quarter 2/Data Mining Project/RecommenderSystem/notebooks/API/Data/MAL/User_anime_facts/pro.pkl","wb") as fp:
#     pickle.dump(d,fp)

In [69]:
import pickle
with open("CFdata.pkl","rb") as fp:
    v = pickle.load(fp)

In [70]:
alpha = v['alpha']
userBiases = v['userBiases'] 
itemBiases = v['itemBiases'] 
userGamma = v['userGamma']
itemGamma =  v['itemGamma'] 

In [71]:
userlist = defaultdict(set)

In [72]:
all_titles = set(mergeduseritem['title'])

In [73]:
for i in reviewsPerUser:
    for j in reviewsPerUser[i]:
        userlist[i].add(j['title'])

In [74]:
def titi(username):
    rem_t = all_titles.difference(userlist[username])
    x = defaultdict()
    for i in rem_t:
        x[i] = prediction(username,i)
    y = dict(sorted(x.items(), key = lambda x: x[1], reverse = True)[:10])
    return y

In [75]:
anime['main_picture'].fillna('{}', inplace = True)
anime['main_picture'] = anime['main_picture'].apply(lambda x:eval(x))

In [76]:

# images = []
# for t in titles:
#     images.append(anime[anime['title']==t]['main_picture'][anime[anime['title']==t]['main_picture'].index.to_numpy()[0]]['medium'])
    
# images

# Content Based Filtering

Data Exploration

In [77]:
df = pd.read_csv('anime.csv')
df.head(5)
df['source'].value_counts(dropna=False)
print(df.info())
print(df.shape)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11299 entries, 0 to 11298
Data columns (total 20 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   anime_id                  11299 non-null  int64 
 1   title                     11299 non-null  object
 2   main_picture              11221 non-null  object
 3   alternative_titles        11299 non-null  object
 4   start_date                10970 non-null  object
 5   end_date                  10186 non-null  object
 6   synopsis                  9442 non-null   object
 7   num_list_users            11299 non-null  int64 
 8   num_scoring_users         11299 non-null  int64 
 9   nsfw                      11299 non-null  object
 10  medium                    11299 non-null  object
 11  status                    11299 non-null  object
 12  num_episodes              11299 non-null  int64 
 13  start_season              9286 non-null   object
 14  source                

Data Cleaning

In [78]:
rem_cols = df.columns.values.tolist() # Saving the list of columns for further use
df  = df[df['synopsis'].notnull()] # Removing all the rows without any synopsis
print(df['synopsis'].isna().sum())
print(df['nsfw'].unique())
df['source'] = df['source'].fillna('other') # Replacing the nan values in source with 'other'
print(df['source'].unique())
df.shape #Shape of the data after removing null values for synopsis

0
['white' 'gray']
['original' 'manga' 'light_novel' 'game' 'visual_novel' '4_koma_manga'
 'novel' 'other' 'picture_book' 'web_manga' 'music' 'radio' 'book'
 'card_game' 'mixed_media' 'web_novel']


(9442, 20)

Feature Engineering

In [79]:
cat = ['nsfw','source'] # List of columns on which one hot encoding needs to be applied
df =df.join(pd.get_dummies(df[cat], sparse=True)) # Applying one hot encoding to the above list of columns

In [80]:
# Applying one hot encoding to the 'Genres' column
df_1 = df[['title','genres']]
df_1['ones'] = 1
df_1['genres'] = df_1['genres'].apply(lambda x:x.strip('][').split(', '))
df_1 = df_1.explode('genres')
pivot = df_1.pivot_table(index = ['title'], columns = ['genres'], values = 'ones')
pivot.fillna(0, inplace = True)
pivot = pivot.reset_index()

/var/folders/60/w5451p953gs03k0nzhrhxt0w0000gn/T/ipykernel_67521/570076148.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_1['ones'] = 1
/var/folders/60/w5451p953gs03k0nzhrhxt0w0000gn/T/ipykernel_67521/570076148.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_1['genres'] = df_1['genres'].apply(lambda x:x.strip('][').split(', '))


In [81]:
#Merging the above dataframe to the original dataframe for genres one hot encoded values.
df = pd.merge(left=df,right = pivot, on = 'title')

In [82]:
# Dropping the unused columns
new_df = df.drop(rem_cols,axis=1)
new_df.head(5)
print(new_df.columns)

Index(['nsfw_gray', 'nsfw_white', 'source_4_koma_manga', 'source_book',
       'source_card_game', 'source_game', 'source_light_novel', 'source_manga',
       'source_mixed_media', 'source_music', 'source_novel', 'source_original',
       'source_other', 'source_picture_book', 'source_radio',
       'source_visual_novel', 'source_web_manga', 'source_web_novel', '',
       ''Action'', ''Adult Cast'', ''Adventure'', ''Anthropomorphic'',
       ''Avant Garde'', ''Award Winning'', ''Boys Love'', ''CGDCT'',
       ''Childcare'', ''Combat Sports'', ''Comedy'', ''Crossdressing'',
       ''Delinquents'', ''Detective'', ''Drama'', ''Ecchi'', ''Educational'',
       ''Erotica'', ''Fantasy'', ''Gag Humor'', ''Girls Love'', ''Gore'',
       ''Gourmet'', ''Harem'', ''Hentai'', ''High Stakes Game'',
       ''Historical'', ''Horror'', ''Idols (Female)'', ''Idols (Male)'',
       ''Isekai'', ''Iyashikei'', ''Josei'', ''Kids'', ''Love Polygon'',
       ''Magical Sex Shift'', ''Mahou Shoujo'', ''Martial

In [83]:
# Pre Processing of 'Synopsis' column
#Tokenizing
def tokenize_words(txt):
  tokenized= word_tokenize(txt)
  return tokenized
#To remove stopwords in text
def remove_stopwords(tokenized_txt):
  cleaned_text_array = [word for word in tokenized_txt if word not in stop_words]
  return cleaned_text_array
#Lammetizing
def word_lemmatizer(tokenized_text_without_stopwords):
  wordnet_lemmatizer = WordNetLemmatizer()
  lemmatized = [wordnet_lemmatizer.lemmatize(word) for word in tokenized_text_without_stopwords]
  return ' '.join(lemmatized)

In [84]:
stop_words = stopwords.words('english')
df['processed_synopsis']=df['synopsis'].apply(tokenize_words)
df['processed_synopsis'] = df['processed_synopsis'].apply(lambda x: remove_stopwords(x))

df['processed_synopsis']= df['processed_synopsis'].apply(lambda x: word_lemmatizer(x))
df[['title','synopsis','processed_synopsis','genres','source','medium']].head(5)

,title,synopsis,processed_synopsis,genres,source,medium
0,Cowboy Bebop,"Crime is timeless. By the year 2071, humanity ...","Crime timeless . By year 2071 , humanity expan...","['Action', 'Adult Cast', 'Award Winning', 'Sci...",original,tv
1,Cowboy Bebop: Tengoku no Tobira,"Another day, another bounty—such is the life o...","Another day , another bounty—such life often u...","['Action', 'Adult Cast', 'Sci-Fi', 'Space']",original,movie
2,Trigun,"Vash the Stampede is the man with a $$60,000,0...","Vash Stampede man $ $ 60,000,000,000 bounty he...","['Action', 'Adult Cast', 'Adventure', 'Sci-Fi'...",manga,tv
3,Witch Hunter Robin,Robin Sena is a powerful craft user drafted in...,Robin Sena powerful craft user drafted STNJ—a ...,"['Action', 'Detective', 'Drama', 'Mystery', 'S...",original,tv
4,Bouken Ou Beet,It is the dark century and the people are suff...,"It dark century people suffering rule devil , ...","['Adventure', 'Fantasy', 'Shounen', 'Supernatu...",manga,tv


In [85]:
# Developing tf-idf matrix for synopsis
df_1 = df['processed_synopsis']
vectorizer = CountVectorizer(analyzer='word', max_features=7000)
x_counts = vectorizer.fit_transform(df_1)
transformer = TfidfTransformer(smooth_idf=False)
x_tfidf = transformer.fit_transform(x_counts)
xtfidf_norm = normalize(x_tfidf, norm='l1', axis=1)
xtfidf_norm.shape

(9442, 7000)

Model

In [86]:
# Calculating cosine similarity scores for both the matrices and averaging them out.
from sklearn.metrics.pairwise import cosine_similarity
cosine_sim_tfidf = cosine_similarity(xtfidf_norm)
cosine_sim_ohe = cosine_similarity(new_df)
cosine_sim = (cosine_sim_tfidf + cosine_sim_ohe)/2

In [87]:
# Recommendation function to get the top 10 similar animes.
anime_index = pd.Series(df.index, index=df.title).drop_duplicates()
def get_recommendation_content(anime_name, similarity=cosine_sim):
    idx = anime_index[anime_name]
    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:11]
    anime_indices = [i[0] for i in sim_scores]
    result = df[['title', 'genres','synopsis']].iloc[anime_indices]
    return result

# Example

Combining content and collaborative filtering for recommendations

Username : imnoodle
Providing collaborative filtering for user BongBong73

In [88]:
user=input("Enter your username : ")
anime_list=titi(user)
titles = list(anime_list.keys())
print(titles)

Enter your username : BongBong73
['Shingeki no Kyojin Season 3 Part 2', 'Steins;Gate', 'Kaguya-sama wa Kokurasetai: Ultra Romantic', 'Kimi no Na wa.', 'Code Geass: Hangyaku no Lelouch R2', 'Kimetsu no Yaiba: Yuukaku-hen', 'Shingeki no Kyojin: The Final Season', 'Kaguya-sama wa Kokurasetai? Tensai-tachi no Renai Zunousen', 'Shingeki no Kyojin: The Final Season Part 2', 'Vinland Saga']


Suppose from the list of animes, imnoodle chooses the anime 'Kimetsu no Yaiba: Yuukaku-hen'. We then provide content based recommendations for the anime 'Kimetsu no Yaiba: Yuukaku-hen'.

In [89]:
get_recommendation_content('Kimetsu no Yaiba: Yuukaku-hen',cosine_sim)

,title,genres,synopsis
8711,Kimetsu no Yaiba: Mugen Ressha-hen,"['Action', 'Fantasy', 'Historical', 'Shounen']",A mysterious string of disappearances on a cer...
7625,Kimetsu no Yaiba Movie: Mugen Ressha-hen,"['Action', 'Fantasy', 'Historical', 'Shounen']",After a string of mysterious disappearances be...
9055,Nokemono-tachi no Yoru,"['Action', 'Anthropomorphic', 'Fantasy', 'Hist...",Wisteria is an orphan girl living in a corner ...
6410,Nanatsu no Taizai Movie 1: Tenkuu no Torawarebito,"['Action', 'Adventure', 'Fantasy', 'Shounen']",In search of a mystical ingredient known as Sk...
7883,Burn the Witch,"['Action', 'Fantasy', 'Shounen']",Although citizens of London view dragons as a ...
8783,Mato Seihei no Slave,"['Action', 'Fantasy', 'Harem', 'Shounen', 'Sup...",When entrances to a different dimension known ...
8903,Ragna Crimson,"['Action', 'Fantasy', 'Shounen']","In a world where dragons rule sky, sea, and la..."
3341,Naruto x UT,"['Action', 'Fantasy', 'Shounen']",All-new animation offered throughout UNIQLO cl...
6221,Saint Seiya: Gold Saints Data File,"['Action', 'Fantasy', 'Shounen']","Short special about Gold Saints, included on D..."
5194,Shounen Sunday CM: InuYasha-hen,"['Action', 'Fantasy', 'Mythology', 'Shounen']","In an age of turmoil, a tenacious couple comes..."
